In [1]:
import os

os.environ['TIKA_LOG_PATH'] = os.getcwd()

In [4]:
!pip install bs4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=d0ee39c66d7a7333091982c088267692d0011daf1dc4b96738239bccfa8a5520
  Stored in directory: /Users/cobra/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [5]:
from glob import glob
from tika import parser
from bs4.element import Comment
from bs4 import BeautifulSoup

TIKA_HOST = 'http://localhost:9998'

In [6]:
pdfs = glob('*/*.pdf')
len(pdfs)

407

In [7]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body, is_html=True):
    if is_html:
        soup = BeautifulSoup(body, 'html.parser')
    else:
        soup = body
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return '\n'.join(t.strip() for t in visible_texts).strip()

In [8]:
def parse_tika(filename):
    raw_xml = parser.from_file(filename, TIKA_HOST, xmlContent=True)
    body = BeautifulSoup(raw_xml['content']).find('body')
    t = []
    for b in body:
        try:
            t_ = text_from_html(b, is_html=False)
            t.append(t_)
        except Exception as e:
            pass
    return t

In [9]:
!ls json

ls: json: No such file or directory


In [10]:
!rm -rf json
!mkdir json

In [11]:
from tqdm import tqdm
import json

def loop(pdfs):
    pdfs, _ = pdfs
    for f in tqdm(pdfs):
        filename = os.path.join('json', f.replace('/', '-') + '.json')
        if os.path.exists(filename):
            continue

        try:
            b = parse_tika(f)
            with open(filename, 'w') as fopen:
                json.dump(b, fopen)
        except Exception as e:
            print(e)

In [18]:
loop((pdfs,0))

  0%|          | 0/407 [00:00<?, ?it/s]/var/folders/1n/9gz34qrs54l8n_v747_v5trw0000gn/T/ipykernel_93141/2886899813.py:14: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  texts = soup.findAll(text=True)
100%|██████████| 407/407 [00:54<00:00,  7.44it/s]


In [19]:
!du -hs json

 39M	json
